## Visualization of the results
---

Now we can make a prediction.

Here we define a workflow to make the model predict based on the existing EOPatches.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import os

import pickle

# Basics of Python data handling and visualization
import itertools
import numpy as np
np.random.seed(42)
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image 

In [ ]:
from sklearn.externals import joblib
from aenum import MultiValueEnum

In [ ]:
# Imports from eo-learn and sentinelhub-py
from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, LoadTask, SaveTask, EOExecutor, ExtractBandsTask, MergeFeatureTask
from eolearn.geometry import VectorToRaster, PointSamplingTask, ErosionTask
from eolearn.io import ExportToTiff
from eolearn.features import LinearInterpolation, SimpleFilterTask, NormalizedDifferenceIndexTask
from sentinelhub import UtmZoneSplitter, BBox, CRS, DataSource

In [ ]:
from osgeo import gdal
#specify the path to the gdal_merge.py script
sys.path.append('./...')
import gdal_merge

In [ ]:
from pathlib import Path
path = Path('./')

### Define EOTasks

In [ ]:
class PredictPatch(EOTask):
    """
    Task to make model predictions on a patch. Provide the model and the feature, 
    and the output names of labels and scores (optional)
    """
    def __init__(self, model, features_feature, predicted_labels_name, predicted_scores_name=None):
        self.model = model
        self.features_feature = features_feature
        self.predicted_labels_name = predicted_labels_name
        self.predicted_scores_name = predicted_scores_name
        
    def execute(self, eopatch):
        ftrs = eopatch[self.features_feature[0]][self.features_feature[1]]
        
        t, w, h, f = ftrs.shape
        ftrs = np.moveaxis(ftrs, 0, 2).reshape(w * h, t * f)
        
        plabels = self.model.predict(ftrs)
        plabels = plabels.reshape(w, h)
        plabels = plabels[..., np.newaxis]
        eopatch.add_feature(FeatureType.MASK_TIMELESS, self.predicted_labels_name, plabels)
        
        if self.predicted_scores_name:
            pscores = self.model.predict_proba(ftrs)
            _, d = pscores.shape
            pscores = pscores.reshape(w, h, d)
            eopatch.add_feature(FeatureType.DATA_TIMELESS, self.predicted_scores_name, pscores)
        
        return eopatch

### Define Tasks and the Workflow

In [ ]:
path_out_sampled = './data/eopatches_sampled'

In [ ]:
# load the trained model from the previous notebook
model_path = './model_SI_LndC.pkl'
model = joblib.load(model_path)

# predict the test labels
#plabels_test = model.predict(features_test)

In [ ]:
# TASK TO LOAD EXISTING EOPATCHES
load = LoadTask(path_out_sampled)

# TASK FOR PREDICTION
predict = PredictPatch(model, (FeatureType.DATA, 'FEATURES'), 'LBL_GBM', 'SCR_GBM')

# TASK FOR SAVING
save = SaveTask(str(path_out_sampled), overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

# TASK TO EXPORT TIFF
export_tiff = ExportToTiff((FeatureType.MASK_TIMELESS, 'LBL_GBM'))
tiff_location = './data/predicted_tiff'
if not os.path.isdir(tiff_location):
    os.makedirs(tiff_location)

workflow = LinearWorkflow(
    load,
    predict,
    export_tiff,
    save
)

In [ ]:
# Get the patchIDs from the previous notebook
#load array
fp = os.path.join(path/'data'/'tile-def/', 'patchIDs.csv')
patchIDs = np.fromfile(fp, dtype=int)
# print the array
print(patchIDs)

### Run the prediction and export to GeoTIFF images

In [ ]:
# create a list of execution arguments for each patch
execution_args = []
for i in patchIDs:
    execution_args.append(
        {
            load: {'eopatch_folder': f'eopatch_{i}'},
            export_tiff: {'filename': f'{tiff_location}/prediction_eopatch_{i}.tiff'},
            save: {'eopatch_folder': f'eopatch_{i}'}
        }
    )

# run the executor
executor = EOExecutor(workflow, execution_args)
executor.run(workers=1, multiprocess=False)
#executor.make_report()

In [ ]:
%%time
# merge with gdal_merge.py (with compression) using bash command magic
# gdal has to be installed on your computer!
!gdal_merge.py -o predicted_tiff/merged_prediction.tiff -co compress=LZW predicted_tiff/prediction_eopatch_*

### To correctly visualize the prediction we need to compare it to the reference

In [ ]:
#load the LandCover classes
class LndC(MultiValueEnum):
    """ 
    Enum class containing LandCover types
    """
    Woodland_and_Forest             = 'Woodland and Forest',  1, '#008000'
    Shrub_and_Grassland             = 'Shrub and Grassland)', 2, '#9370DB'
    Water                           = 'Water',                3, '#000080'
    Mines                           = 'Mines',                4, '#8B0000'
    Wetlands                        = 'Wetlands',             5, '#00CED1'
    Bare_Non_Vegetated              = 'Bare Non-Vegetated',   6, '#FFFACD'
    Cultivated_Commercial           = 'Cultivated_Commercial',7, '#DC143C'
    Fallow_land                     = 'Fallow land',          8, '#F08080'
    Formal_Residential              = 'Formal Residential',   9, '#FFA500'
    Informal_Residential            = 'Informal Residential', 10, '#FF69B4'
    Village                         = 'Village',              11, '#FF8C00'
    Smallholding                    = 'Smallholding',         12, '#DDA0DD'
    Urban_Recreation                = 'Urban Recreation',     13, '#7FFF00'
    Commercial                      = 'Commercial',           14, '#FF005D'
    Industrial                      = 'Industrial',           15, '#FFFF00'
    Major_Road_and_Rail             = 'Major Road and Rail',  16, '#FFD700'
    
    @property
    def id(self):
        """ Returns an ID of an enum type
        :return: An ID
        :rtype: int
        """
        return self.values[1]

    @property
    def color(self):
        """ Returns class color
        :return: A color in hexadecimal representation
        :rtype: str
        """
        return self.values[2]

def get_bounds_from_ids(ids):
    bounds = []
    for i in range(len(ids)):
        if i < len(ids) - 1:
            if i == 0:
                diff = (ids[i + 1] - ids[i]) / 2
                bounds.append(ids[i] - diff)
            diff = (ids[i + 1] - ids[i]) / 2
            bounds.append(ids[i] + diff)
        else:
            diff = (ids[i] - ids[i - 1]) / 2
            bounds.append(ids[i] + diff)
    return bounds 

# Reference colormap things
lulc_bounds = get_bounds_from_ids([x.id for x in LndC])
lulc_cmap = ListedColormap([x.color for x in LndC], name="lulc_cmap")
lulc_norm = BoundaryNorm(lulc_bounds, lulc_cmap.N)

### Visualise the prediction

In [ ]:
#path_out_sampled = './eopatches_sampled'

fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(20, 25))

for i, ax in zip(patchIDs, axes.flatten()):
    eopatch = EOPatch.load(f'{path_out_sampled}/eopatch_{i}', lazy_loading=True)
    #ax = axs[i//5][i%5]
    im = ax.imshow(eopatch.mask_timeless['LBL_GBM'].squeeze(), cmap=lulc_cmap, norm=lulc_norm)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    del eopatch

fig.subplots_adjust(wspace=0, hspace=0)

cb = fig.colorbar(im, ax=axes.ravel().tolist(), orientation='horizontal', pad=0.01, aspect=40)
cb.ax.tick_params(labelsize=20) 
cb.set_ticks([entry.id for entry in LndC])
cb.ax.set_xticklabels([entry.name for entry in LndC], rotation=45, fontsize=15)
fig.savefig(f'figs/lndC_Prediction.png', bbox_inches='tight')
plt.show()

![image](figs/LndC_Prediction.png)

### Visual inspection of patches

Its possible to 'really' inspect the predictions. 

Specific or random subsets of ```Patches```, via ```idx``` can be chosen, where prediction and ground truth are compared. ```Inspect_size``` can be set to zoom. For visual aid the mask of differences and the true color image are also provided.

The image below represents the bottom right corner of the far left middle row ```EOPatch``` idx = 71 - in the plot above.

In [ ]:
# Draw the Reference map
fig = plt.figure(figsize=(20, 20))

#manual select
idx = 71
w_min = 215
h_min = 87

#this sets the zoom
inspect_size = 400

#uncomment to randomly select
#idx = np.random.choice(range(len(patchIDs)))

eopatch = EOPatch.load(f'{path_out_sampled}/eopatch_{idx}', lazy_loading=True)

w, h = eopatch.mask_timeless['LndC'].squeeze().shape

#uncomment if random select is chosen
#w_min = np.random.choice(range(w - inspect_size))
#h_min = np.random.choice(range(h - inspect_size))

ax = plt.subplot(2, 2, 1)
plt.imshow(eopatch.mask_timeless['LndC'].squeeze()[w_min: w_min + inspect_size, h_min : h_min + inspect_size],
           cmap=lulc_cmap, norm=lulc_norm)
plt.xticks([])
plt.yticks([])
ax.set_aspect("auto")
plt.title('Ground Truth', fontsize=20)

ax = plt.subplot(2, 2, 2)
plt.imshow(eopatch.mask_timeless['LBL_GBM'].squeeze()[w_min: w_min + inspect_size, h_min: h_min + inspect_size],
           cmap=lulc_cmap, norm=lulc_norm)
plt.xticks([])
plt.yticks([])
ax.set_aspect("auto")
plt.title('Prediction', fontsize=20)

ax = plt.subplot(2, 2, 3)
mask = eopatch.mask_timeless['LBL_GBM'].squeeze() != eopatch.mask_timeless['LndC'].squeeze()
plt.imshow(mask[w_min: w_min + inspect_size, h_min: h_min + inspect_size], cmap='gray')
plt.xticks([])
plt.yticks([]);
ax.set_aspect("auto")
plt.title('Difference', fontsize=20)

ax = plt.subplot(2, 2, 4)
image = np.clip(eopatch.data['FEATURES'][0][..., [2, 1, 0]] * 3.5, 0, 1)
plt.imshow(image[w_min: w_min + inspect_size, h_min: h_min + inspect_size])
plt.xticks([])
plt.yticks([]);
ax.set_aspect("auto")
plt.title('True Color', fontsize=20)

fig.subplots_adjust(wspace=0.1, hspace=0.1)
fig.savefig(f'figs/Prediction_comparison_02.png', bbox_inches='tight')

![image](figs/Prediction_comparison_02.png)

## There we have it.